In [110]:
import pandas as pd
import numpy as np
import sklearn
import os

utilities_path = "../../scripts/utilities/"
import sys
if utilities_path not in sys.path:
    sys.path.append(utilities_path)

In [112]:
from utilities import *

# Define Functions

In [73]:
help(data.UsageBand.cat.set_categories)

Help on method set_categories in module pandas.core.accessor:

set_categories(*args, **kwargs) method of pandas.core.arrays.categorical.CategoricalAccessor instance
    Set the categories to the specified new_categories.
    
    `new_categories` can include new categories (which will result in
    unused categories) or remove old categories (which results in values
    set to NaN). If `rename==True`, the categories will simple be renamed
    (less or more items than in old categories will result in values set to
    NaN or in unused categories respectively).
    
    This method can be used to perform more than one action of adding,
    removing, and reordering simultaneously and is therefore faster than
    performing the individual steps via the more specialised methods.
    
    On the other hand this methods does not do checks (e.g., whether the
    old categories are included in the new categories on a reorder), which
    can result in surprising changes, for example when using s

In [63]:
is_string_dtype(data.UsageBand)

True

# Constant Variable Declaration

In [46]:
path = "../"

In [47]:
train_path = f"{path}Train.csv"
test_path = f"{path}Valid.csv"

In [48]:
train_path

'../Train.csv'

In [49]:
## Create codes for doing kaggle competition and save it separately
## import from the module for my processes (Use symlinks if necessary)
## Make code for evaluating results on valid set and show result

In [76]:
data = pd.read_csv(train_path, parse_dates= ["saledate"], low_memory= False)
test_data = pd.read_csv(test_path, parse_dates= ["saledate"], low_memory= False)

In [53]:
display_all(data.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


In [82]:
data.Travel_Controls.unique()

[NaN, None or Unspecified, Differential Steer, Lever, Finger Tip, 2 Pedal, Pedal, 1 Speed]
Categories (7, object): [1 Speed < 2 Pedal < Differential Steer < Finger Tip < Lever < None or Unspecified < Pedal]

In [81]:
test_data.Travel_Controls.unique()

array([nan, 'None or Unspecified', 'Lever', 'Finger Tip',
       'Differential Steer', '1 Speed', 'Pedal', '2 Pedal'], dtype=object)

In [ ]:
#### convert categorical variables to numerical
#### take care of missing values (replace by median or custom value, make rows showing where missing was)

In [60]:
## converting y to log scale to match evaluation metric
data.log_price = np.log(data.SalePrice)

In [146]:
data["Transmission"] = data.Transmission.str.replace("AutoShift","Autoshift")
train_cats(data)
data.UsageBand.cat.set_categories(['High','Medium','Low'], ordered = True, inplace = True)
data.ProductSize.cat.set_categories(['Mini','Small',"Compact", "Medium","Large / Medium","Large"], ordered = True, inplace = True)
data.Drive_System.cat.set_categories(["No", "Two Wheel Drive","Four Wheel Drive","All Wheel Drive"], ordered = True, inplace = True)
data.Enclosure.cat.set_categories(['None or Unspecified','NO ROPS','OROPS', "EROPS", "EROPS AC","EROPS w AC"], ordered = True, inplace = True)
data.Blade_Width.cat.set_categories(['None or Unspecified',"<12'","12'","13'","14'","16'"], ordered = True, inplace = True)
data.Ripper.cat.set_categories(['None or Unspecified', 'High','Medium','Low'], ordered = True, inplace = True)
data.UsageBand.cat.set_categories(['High','Medium','Low'], ordered = True, inplace = True)
apply_cats(test_data, data)

In [151]:
data.Ripper.unique()

[NaN, None or Unspecified, Yes, Single Shank, Multi Shank]
Categories (4, object): [Multi Shank < None or Unspecified < Single Shank < Yes]

In [148]:
data.Blade_Width.unique()

[NaN, None or Unspecified, 12', 14', 13', 16', <12']
Categories (6, object): [None or Unspecified < <12' < 12' < 13' < 14' < 16']

In [149]:
test_data.Blade_Width.unique()

[NaN, 14', 12', None or Unspecified, 16', 13', <12']
Categories (6, object): [None or Unspecified < <12' < 12' < 13' < 14' < 16']

In [104]:
## Since it is ordered, ensuring order is correct for all variables
for n,c in data.items():
    if is_categorical_dtype(c):
        if len(c.cat.categories) < 10:
            print(n, c.cat.categories)
            
## check order for 
'''
UsageBand
ProductSize
Drive_System
Enclosure_Index
Transmission Index (change autoshift and autoshift to same)
Blade_Width Index
Enclosure_Type Index
Ripper Index
Tip_Control Index
Coupler Index
Hydraulics_Flow Index
Thumb Index
Pattern_Changer Index
Grouser_Type Index
'''


UsageBand Index(['High', 'Low', 'Medium'], dtype='object')
ProductSize Index(['Compact', 'Large', 'Large / Medium', 'Medium', 'Mini', 'Small'], dtype='object')
ProductGroup Index(['BL', 'MG', 'SSL', 'TEX', 'TTT', 'WL'], dtype='object')
ProductGroupDesc Index(['Backhoe Loaders', 'Motor Graders', 'Skid Steer Loaders',
       'Track Excavators', 'Track Type Tractors', 'Wheel Loader'],
      dtype='object')
Drive_System Index(['All Wheel Drive', 'Four Wheel Drive', 'No', 'Two Wheel Drive'], dtype='object')
Enclosure Index(['EROPS', 'EROPS AC', 'EROPS w AC', 'NO ROPS', 'None or Unspecified',
       'OROPS'],
      dtype='object')
Forks Index(['None or Unspecified', 'Yes'], dtype='object')
Pad_Type Index(['Grouser', 'None or Unspecified', 'Reversible', 'Street'], dtype='object')
Ride_Control Index(['No', 'None or Unspecified', 'Yes'], dtype='object')
Stick Index(['Extended', 'Standard'], dtype='object')
Transmission Index(['AutoShift', 'Autoshift', 'Direct Drive', 'Hydrostatic',
       'None